In [1]:
# Dataset and Connection to SAP HANA 

from hana_ml import dataframe
from hana_ml.algorithms.pal.neural_network import MLPClassifier, MLPRegressor
from data_load_utils import Settings
from hana_ml.dataframe import create_dataframe_from_pandas
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# save load_iris() sklearn dataset to iris
# if you'd like to check dataset type use: type(load_iris())
# if you'd like to view list of attributes use: dir(load_iris())
iris = load_iris()

# np.c_ is the numpy concatenate function
# which is used to concat iris['data'] and iris['target'] arrays 
# for pandas column argument: concat iris['feature_names'] list
# and string list (in this case one string); you can make this anything you'd like..  
# the original dataset would probably call this ['Species']
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
hana_df = create_dataframe_from_pandas(connection_context, data, 'IRIS', force=True)

hana_df.head(10).collect()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.70it/s]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


In [2]:
from hana_ml.model_storage import ModelStorage
#from hana_ml.model_storage_services import ModelSavingServices

model = MLPRegressor(hidden_layer_size=[10,], 
                     activation='TANH', 
                     output_activation='TANH', 
                     learning_rate=0.01, 
                     momentum=0.001)
model.fit(hana_df, label='target')

# Creates an object called model_storage which must use the same connection with the model
model_storage = ModelStorage(connection_context=connection_context)
model_storage.clean_up()
# Saves the model for the first time
model.name = 'Model A'  # The model name is mandatory
model.version = 1
model_storage.save_model(model=model)

# Lists models
model_storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,STORAGE_TYPE,MODEL_STORAGE_VER,SCHEDULE
0,Model A,1,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""activation"": ""TANH"", ""a...",2021-07-07 11:56:01,default,1,"{""schedule"": {""status"": ""inactive"", ""schedule_..."


In [3]:
# save the model for the second time with 'replace' option and expect to have a model called 'Model A' and version is 1
model_storage.save_model(model=model, if_exists='replace')
model_storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,STORAGE_TYPE,MODEL_STORAGE_VER,SCHEDULE
0,Model A,1,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""activation"": ""TANH"", ""a...",2021-07-07 11:56:09,default,1,"{""schedule"": {""status"": ""inactive"", ""schedule_..."


In [4]:
# save the model for the second time with 'upgrade' option and expect to have two model called 'Model A' 
# and version is 1 and 2
model_storage.save_model(model=model, if_exists='upgrade')
print(model.version)
model_storage.list_models()

2


,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,STORAGE_TYPE,MODEL_STORAGE_VER,SCHEDULE
0,Model A,1,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""activation"": ""TANH"", ""a...",2021-07-07 11:56:09,default,1,"{""schedule"": {""status"": ""inactive"", ""schedule_..."
1,Model A,2,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""activation"": ""TANH"", ""a...",2021-07-07 11:56:15,default,1,"{""schedule"": {""status"": ""inactive"", ""schedule_..."


In [5]:
# See the detail of model
model_storage.list_models('Model A', 2)['JSON'].iloc[0]

'{"model_attributes": {"activation": "TANH", "activation_options": null, "output_activation": "TANH", "output_activation_options": null, "hidden_layer_size": [10], "hidden_layer_size_options": null, "max_iter": null, "training_style": "stochastic", "learning_rate": 0.01, "momentum": 0.001, "batch_size": null, "normalization": null, "weight_init": null, "categorical_variable": null, "resampling_method": null, "evaluation_metric": null, "fold_num": null, "repeat_times": null, "search_strategy": null, "random_search_times": null, "random_state": null, "timeout": null, "progress_indicator_id": null, "thread_ratio": null, "param_range": null, "param_values": null}, "artifacts": {"schema": "PAL_TEST", "model_tables": "HANAML_MODEL_A_2_CLASS_HANA_ML_ALGORITHMS_PAL_NEURAL_NETWORK_MLPREGRESSOR_MODELS", "library": "PAL"}}'

In [6]:
# Load the model
new_model = model_storage.load_model(name='Model A', version=1)

# Predict with the loaded model
test = pd.DataFrame()
test['id'] = [1]
test['sepal length (cm)'] = [4.2]
test['sepal width (cm)'] = [3.2]
test['petal length (cm)'] = [1.2]
test['petal width (cm)'] = [0.2]

test_df = create_dataframe_from_pandas(connection_context, test, 'IRIS_TEST', force=True)
new_model.predict(test_df, key='id').collect()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.75it/s]


,id,TARGET,VALUE
0,1,target,0.0


In [7]:
new_model.model_.select_statement

'SELECT * FROM "PAL_TEST"."HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_NEURAL_NETWORK_MLPREGRESSOR_MODELS"'

In [8]:
# enable persistent memory of a model 'Model A' with version 1
model_storage.enable_persistent_memory('Model A', 1)

In [9]:
# see the Runtime data of column tables M_CS_TABLES
# LOADED: If the table is loaded in memory
# PERSISTENT_MEMORY: Flag to indicate that new main part will be written to disk during table delta merge unless requested differently
import json
connection_context.sql("SELECT TABLE_NAME, LOADED, PERSISTENT_MEMORY FROM M_CS_TABLES WHERE TABLE_NAME='{}'".format(json.loads(model_storage.list_models('Model A',1)["JSON"][0])["artifacts"]["model_tables"])).collect()

,TABLE_NAME,LOADED,PERSISTENT_MEMORY
0,HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_...,PARTIALLY,TRUE


In [10]:
# Unload from the memory
model_storage.unload_from_memory("Model A", 1, persistent_memory='retain')

In [11]:
# see the Runtime data of column tables M_CS_TABLES
connection_context.sql("SELECT TABLE_NAME, LOADED, PERSISTENT_MEMORY FROM M_CS_TABLES WHERE TABLE_NAME='{}'".format(json.loads(model_storage.list_models('Model A',1)["JSON"][0])["artifacts"]["model_tables"])).collect()

,TABLE_NAME,LOADED,PERSISTENT_MEMORY
0,HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_...,NO,TRUE


In [12]:
model_storage.disable_persistent_memory('Model A', 1)

In [13]:
connection_context.sql("SELECT TABLE_NAME, LOADED, PERSISTENT_MEMORY FROM M_CS_TABLES WHERE TABLE_NAME='{}'".format(json.loads(model_storage.list_models('Model A',1)["JSON"][0])["artifacts"]["model_tables"])).collect()

,TABLE_NAME,LOADED,PERSISTENT_MEMORY
0,HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_...,PARTIALLY,FALSE


In [14]:
model_storage.load_into_memory("Model A", 1)

In [15]:
connection_context.sql("SELECT TABLE_NAME, LOADED, PERSISTENT_MEMORY FROM M_CS_TABLES WHERE TABLE_NAME='{}'".format(json.loads(model_storage.list_models('Model A',1)["JSON"][0])["artifacts"]["model_tables"])).collect()

,TABLE_NAME,LOADED,PERSISTENT_MEMORY
0,HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_...,FULL,FALSE


In [16]:
# Delete a model
model_storage.delete_model('Model A', 1)

In [17]:
model_storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,STORAGE_TYPE,MODEL_STORAGE_VER,SCHEDULE
0,Model A,2,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""activation"": ""TANH"", ""a...",2021-07-07 11:56:15,default,1,"{""schedule"": {""status"": ""inactive"", ""schedule_..."


In [18]:
model_storage.list_models(name='Model A')

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,STORAGE_TYPE,MODEL_STORAGE_VER,SCHEDULE
0,Model A,2,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""activation"": ""TANH"", ""a...",2021-07-07 11:56:15,default,1,"{""schedule"": {""status"": ""inactive"", ""schedule_..."


In [19]:
# Detele models with specified name and a specified range of time
start_time = '2019-07-27 07:49:48'
end_time = '2021-07-29 10:26:47'
model_storage.delete_models('Model A', start_time=start_time, end_time=end_time)
model_storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,STORAGE_TYPE,MODEL_STORAGE_VER,SCHEDULE


In [20]:
# Clean up all models at once
model_storage.clean_up()

In [21]:
# Close the connection to SAP HANA
connection_context.close()